In [ ]:
from valentine import valentine_match
from valentine.algorithms import Coma
from valentine.algorithms import Cupid
import os
import multiprocessing as mp
from multiprocessing import Pool
import time
import pandas as pd
from compute_match import *

In [ ]:
path = "../json_datasets"

data_frames = {}
for file in os.listdir(path):
    filepath = f"{path}/{file}"
    data_frames[file] = (pd.read_json(filepath))

In [ ]:
keys = list(data_frames.keys())
tuples = []
# dfs_len = len(keys)
dfs_len = 5
for i in range(dfs_len):
    for j in range(i + 1, dfs_len):
        dfl = data_frames[keys[i]]
        dfr = data_frames[keys[j]]
        tuples.append((keys[i], keys[j], dfl, dfr))

In [ ]:
start_time = time.perf_counter()
with Pool() as pool:
    result = pool.map(calculate_match, tuples)
print("Program finished!")
finish_time = time.perf_counter()

In [ ]:
columns = ["table_1", "table_2", "column_table_1", "column_table_2", "match_value"]
matches = pd.DataFrame(columns=columns)

for res in result:
    for key in res[2].keys():
        d = dict()
        d["table_1"] = res[0]
        d["table_2"] = res[1]
        d["column_table_1"] = key[0][1]
        d["column_table_2"] = key[1][1]
        d["match_value"] = res[2][key]
        matches = pd.concat([matches, pd.DataFrame(d, index=[0])], ignore_index=True)
print(matches)

In [ ]:
table_to_matches = dict()
for table in matches['table_1']:
    if table_to_matches.get(table) == None:
        table_to_matches[table] = 0
    table_to_matches[table] += 1
print(table_to_matches)

In [ ]:
pd.DataFrame(matches).to_csv("./instance_matches.csv")